## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-19-20-58-53 +0000,nyt,Live Updates: Former Prince Andrew Is Released...,https://www.nytimes.com/live/2026/02/19/world/...
1,2026-02-19-20-58-14 +0000,nyt,What to Know About Former Prince Andrew’s Arre...,https://www.nytimes.com/2026/02/19/world/europ...
2,2026-02-19-20-57-56 +0000,nypost,Tourists tie pules of trash — including tampon...,https://nypost.com/2026/02/19/us-news/tourists...
3,2026-02-19-20-56-21 +0000,wapo,Trump appears ready to attack Iran as U.S. str...,https://www.washingtonpost.com/national-securi...
4,2026-02-19-20-53-50 +0000,nyt,South Korean Ex-Leader Is Sentenced to Life in...,https://www.nytimes.com/live/2026/02/19/world/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2475/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
25,trump,44
9,epstein,25
4,andrew,24
29,iran,24
3,prince,21


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
41,2026-02-19-20-12-03 +0000,nypost,Trump expresses sympathy for the royal family ...,https://nypost.com/2026/02/19/us-news/trump-ex...,148
28,2026-02-19-20-28-03 +0000,nyt,The police are investigating former Prince And...,https://www.nytimes.com/live/2026/02/19/world/...,130
214,2026-02-19-11-48-00 +0000,wsj,"As Trump’s Board of Peace meets Thursday, can ...",https://www.wsj.com/politics/trump-moves-towar...,128
59,2026-02-19-19-31-44 +0000,nypost,Ex-Prince Andrew looks aghast in first photos ...,https://nypost.com/2026/02/19/world-news/ex-pr...,119
0,2026-02-19-20-58-53 +0000,nyt,Live Updates: Former Prince Andrew Is Released...,https://www.nytimes.com/live/2026/02/19/world/...,119


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
41,148,2026-02-19-20-12-03 +0000,nypost,Trump expresses sympathy for the royal family ...,https://nypost.com/2026/02/19/us-news/trump-ex...
273,92,2026-02-19-07-49-29 +0000,nypost,Former South Korean President Yoon sentenced t...,https://nypost.com/2026/02/19/world-news/forme...
214,74,2026-02-19-11-48-00 +0000,wsj,"As Trump’s Board of Peace meets Thursday, can ...",https://www.wsj.com/politics/trump-moves-towar...
170,67,2026-02-19-14-39-21 +0000,nypost,White House warns Iran it’d be ‘very wise’ to ...,https://nypost.com/2026/02/19/us-news/white-ho...
256,57,2026-02-19-09-58-53 +0000,nypost,Rep. Lauren Boebert’s son Tyler slapped with a...,https://nypost.com/2026/02/19/us-news/lauren-b...
207,46,2026-02-19-12-11-49 +0000,nypost,UK Prime Minister Keir Starmer warned ‘nobody ...,https://nypost.com/2026/02/19/world-news/uk-pr...
285,46,2026-02-19-05-01-09 +0000,nyt,Policy Flip-Flops Hurt the British Leader. The...,https://www.nytimes.com/2026/02/19/world/europ...
82,41,2026-02-19-18-51-27 +0000,nypost,Lake Tahoe avalanche victim was married to sea...,https://nypost.com/2026/02/19/us-news/tahoe-av...
294,41,2026-02-19-03-49-52 +0000,nypost,Husband of Rep. Tony Gonzales aide who fatally...,https://nypost.com/2026/02/18/us-news/husand-o...
154,41,2026-02-19-15-39-00 +0000,wsj,"The U.S. trade deficit jumped in December, the...",https://www.wsj.com/economy/trade/u-s-trade-de...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
